In [1]:
import pickle
import rospy
import baxter_interface
import os.path as path
import copy
from tqdm import tqdm_notebook as tqdmn
import matplotlib.pyplot as plt
import itertools
import numpy as np
import time

In [2]:
# initialize ros node
rospy.init_node('trajectory_player')
limb = baxter_interface.Limb('right')

In [3]:
# define relevant variables
PLAYBACK_MODE = 'bend'
REACH_TIME = 0.05
file_seed = path.expanduser('~/data/moveit_data/')
if PLAYBACK_MODE == 'bend':
    moveit_file = file_seed + 'bend_dof_'
else:
    file_seed = file_seed + 'full_dof_'
limb.set_joint_position_speed(0.75)

In [ ]:
# in a loop load all the plans and replay
joint_angles = {}
err = []
xaxis = []
# observations = []
for file_iter in tqdmn(xrange(50), desc='Files read:'):
    plan = pickle.load(open(moveit_file + str(file_iter) + '.pkl', 'rb'))
    ctr = 0
    err.append(dict())
    err[file_iter] = {'s1':[], 'e1': [], 'w1':[]}
    obs = []
    for ctr in tqdmn(xrange(len(plan)), desc='Waypoints achieved:'):
        if ctr == 0:
            joint_names = copy.deepcopy(plan[ctr])
#             print joint_names
        else:
            current_velocities = limb.joint_velocities()
            current_torques = limb.joint_efforts()
            for (i, joint) in enumerate(joint_names):
                joint_angles[joint] = plan[ctr][i]
            if ctr == 1:
                limb.move_to_joint_positions(joint_angles)
            else:
                start_time = time.time()
                while ((time.time() - start_time) < REACH_TIME):
                    limb.set_joint_positions(joint_angles)
            measured_angles = limb.joint_angles()
            err[file_iter]['e1'].append(measured_angles['right_e1']-joint_angles['right_e1'])
            err[file_iter]['s1'].append(measured_angles['right_s1']-joint_angles['right_s1'])
            err[file_iter]['w1'].append(measured_angles['right_w1']-joint_angles['right_w1'])
            # append all dicts to a list
#     observations.append(obs)
#     xaxis.append(xrange(len(plan)-1))
#     plt.plot(xaxis, err['e1']*180/np.pi, 'b')
#     plt.plot(xaxis, err['s1']*180/np.pi, 'r')
#     plt.plot(xaxis, err['w1']*180/np.pi, 'g')
#     plt.show()

In [ ]:
# plot commanded trajectory and error
joint_angles = {}
for file_iter in tqdmn(xrange(2), desc='Files read:'):
    plan = pickle.load(open(moveit_file + str(file_iter) + '.pkl', 'rb'))
    s1 = [row[1] for row in plan[1:]]
    e1 = [row[3] for row in plan[1:]]
    w1 = [row[5] for row in plan[1:]]
    plt.figure(file_iter)
    plt.plot(np.array(s1)*180/np.pi, 'b')
    plt.plot(np.array(e1)*180/np.pi, 'r')
    plt.plot(np.array(w1)*180/np.pi, 'g')
    plt.plot(np.array(err[file_iter]['e1'])*180./np.pi, 'b--')
    plt.plot(np.array(err[file_iter]['s1'])*180./np.pi, 'r--')
    plt.plot(np.array(err[file_iter]['w1'])*180./np.pi, 'g--')
    plt.legend(['e1', 's1', 'w1'])
    plt.show()

plt.figure(0)
plt.plot(np.array(err[0]['e1'])*180./np.pi, 'b')
plt.plot(np.array(err[0]['s1'])*180./np.pi, 'r')
plt.plot(np.array(err[0]['w1'])*180./np.pi, 'g')
plt.legend(['e1', 's1', 'w1'])
plt.show()
plt.figure(1)
plt.plot(np.array(err[1]['e1'])*180./np.pi, 'b')
plt.plot(np.array(err[1]['s1'])*180./np.pi, 'r')
plt.plot(np.array(err[1]['w1'])*180./np.pi, 'g')
plt.legend(['e1', 's1', 'w1'])
plt.show()